# Silver as a techno/biophysical constraint for a large-scale development of Solar Photovoltaics? Current trends and data quality

We present here the code underpinning the assessment of the silver constraint along with the rationale behind the steps undertaken

<!-- dom:FIGURE: [figs/hypersphere.png, width=400 frac=0.7] Ratio of hypersphere volume to hypercube volume.  -->
<!-- begin figure -->

<p>Ratio of hypersphere volume to hypercube volume.</p>
<img src="figs/hypersphere.png" width=400>

<!-- end figure -->